In [66]:
import numpy as np
import pandas as pd
import plotly.express as px
import random as rd
import json
import math
from scipy.stats import truncnorm

In [130]:
def get_norm_dist(mean=0.5, sd=0.25, low=0, upp=1):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def generate_prob(n):
    prob = get_norm_dist()
    return prob.rvs(n)

def infected(probability):
    rand = np.random.rand()
    if(rand <= probability):
        return True
    return False

In [152]:
class Simulation():
    def __init__(self, nodes, timesteps, radius):
        self.population = nodes
        self.susceptibility = generate_prob(nodes)
        self.time = timesteps*4
        self.radius = radius
        self.hstatus = np.array([])
        self.infected = np.zeros(self.population)
        self.generate_walk()
        
    def generate_walk(self):
        self.generate_nodes()
        for t in range(self.time-1):
            for i in range(self.population):
                x = rd.randint(0,2)
                y = rd.randint(0,2)
                if x == 1:
                    if self.x[i]+2<=50:
                        self.x[i]+=2
                elif x == 2:
                    if self.x[i]-2>=0:
                        self.x[i]-=2
                if y == 1:
                    if self.y[i]+2<=50:
                        self.y[i]+=2
                elif y == 2:
                    if self.y[i]-2>=0:
                        self.y[i]-=2
                # Check if node is infected to update time-series
                if self.hstatus[i] =='Infected':
                    self.spread_contagion(i)
                    self.infected[i] += 0.25
                if self.infected[i] == 14:
                    self.infected[i] = -1
                    self.hstatus[i] = 'Recovered'
                   
            self.distance = np.array([math.sqrt(self.x[i]**2+self.y[i]**2) for i in range(self.population-1)])
            data = {'id': np.arange(self.population), 'timestep': np.ones(self.population)*(t+1), 'x_pos': self.x, 'y_pos': self.y,
                   'health_status': self.hstatus, 'days_infected': self.infected}
            self.df = pd.concat([self.df, pd.DataFrame(data)])
    
    def generate_nodes(self):
        self.x = np.random.rand(self.population)*50
        self.y = np.random.rand(self.population)*50
        self.hstatus = np.append(self.hstatus, np.array(["Susceptible" for x in range(self.population-1)]))
        self.hstatus = np.append(self.hstatus, 'Infected')
        data = {'id': np.arange(self.population), 'timestep':np.zeros(self.population), 'x_pos': self.x, 'y_pos': self.y, 
                'health_status': self.hstatus, 'days_infected': self.infected}
        self.df = pd.DataFrame(data)
        
    def spread_contagion(self, index):
        for i in range(self.population):
                if self.hstatus[i] != 'Infected':
                    distance = abs(math.sqrt((self.x[index] - self.x[i])**2+(self.y[index] - self.y[i])**2))
                    if distance <= self.radius:
                        if infected(self.susceptibility[i]):
                            self.hstatus[i] = 'Infected'

    def view_df(self):
        return self.df
        
    def run(self):
        self.generate_nodes()
        self.generate_walk()

In [153]:
s = Simulation(50, 30, 3)
s.view_df()

,id,timestep,x_pos,y_pos,health_status,days_infected
0,0,0.0,30.671958,46.468831,Susceptible,0.0
1,1,0.0,19.126150,3.664958,Susceptible,0.0
2,2,0.0,1.987212,10.692715,Susceptible,0.0
3,3,0.0,20.234783,1.082929,Susceptible,0.0
4,4,0.0,18.227538,16.021091,Susceptible,0.0
5,5,0.0,32.408429,19.194297,Susceptible,0.0
6,6,0.0,6.955386,25.366714,Susceptible,0.0
7,7,0.0,44.855246,31.076195,Susceptible,0.0
8,8,0.0,32.897105,38.456135,Susceptible,0.0
9,9,0.0,32.856898,41.564893,Susceptible,0.0


In [151]:
px.scatter(s.view_df(), x="x_pos", y="y_pos", animation_frame="timestep", animation_group="id", color="health_status", range_x=[0,50], range_y=[0,50])

In [126]:
s.view_df()

,id,timestep,x_pos,y_pos,health_status,days_infected
0,0,0.0,5.280184,36.060971,Infected,0.0
1,1,0.0,6.896798,19.784006,Susceptible,0.0
2,2,0.0,0.336211,28.123021,Susceptible,0.0
3,3,0.0,0.083544,3.572688,Susceptible,0.0
4,4,0.0,13.033291,16.428471,Susceptible,0.0
5,5,0.0,22.804087,16.744407,Susceptible,0.0
6,6,0.0,22.174324,46.750271,Susceptible,0.0
7,7,0.0,1.702630,40.460197,Susceptible,0.0
8,8,0.0,8.337111,24.822986,Susceptible,0.0
9,9,0.0,11.645842,35.115508,Susceptible,0.0
